In [1]:
import pandas as pd

In [2]:
print("Reading from PLACE starts")
df_PLACE_ins = pd.read_parquet('/export/usuarios_ml4ds/lbartolome/NextProcurement/sproc/place_feb_21/insiders.parquet')
df_PLACE_out = pd.read_parquet('/export/usuarios_ml4ds/lbartolome/NextProcurement/sproc/place_feb_21/outsiders.parquet')
df_PLACE_min = pd.read_parquet('/export/usuarios_ml4ds/lbartolome/NextProcurement/sproc/place_feb_21/minors.parquet')

Reading from PLACE starts


In [4]:
df_PLACE_out['origin'] = 'out'
df_PLACE_ins['origin'] = 'ins'
df_PLACE_min['origin'] = 'min'

def unify_colname(col):
    return ".".join([el for el in col if el])

df_PLACE_ins = df_PLACE_ins.rename(columns={"id":"place_id"})
df_PLACE_min = df_PLACE_min.rename(columns={"id":"place_id"})
df_PLACE_out = df_PLACE_out.rename(columns={"id":"place_id"})

df_PLACE_out.columns = [unify_colname(col) for col in df_PLACE_out.columns]
df_PLACE_ins.columns = [unify_colname(col) for col in df_PLACE_ins.columns]
df_PLACE_min.columns = [unify_colname(col) for col in df_PLACE_min.columns]

In [5]:
columns_keep_ins_min = [
    #'ContractFolderStatus.ContractFolderID', # Expediente
    'ContractFolderStatus.ProcurementProject.RequiredCommodityClassification.ItemClassificationCode', # CPV
    'ContractFolderStatus.ProcurementProject.RealizedLocation.CountrySubentityCode', # Country NUTS
    'ContractFolderStatus.LocatedContractingParty.ParentLocatedParty.PartyName.Name', # Ubicación orgánica
    'ContractFolderStatus.ProcurementProject.BudgetAmount.EstimatedOverallContractAmount', # Presupuesto
    'ContractFolderStatus.TenderResult.SMEAwardedIndicator', # Es pyme
    'ContractFolderStatus.ContractFolderID', #Expediente
    'title', # PLACE title (objective)
    'ContractFolderStatus.ValidNoticeInfo.AdditionalPublicationStatus.AdditionalPublicationDocumentReference.IssueDate', # Date
    'origin',
    'place_id',
    'link'
]
# outsiders
columns_keep_out = [
    #'ContractFolderStatus.ContractFolderID', # Expediente
    'ContractFolderStatus.ProcurementProject.RequiredCommodityClassification.ItemClassificationCode', #CPV
    'ContractFolderStatus.ProcurementProject.RealizedLocation.CountrySubentityCode', # Country NUTS
    'ContractFolderStatus.LocatedContractingParty.ParentLocatedParty.PartyName.Name', # Ubicación orgánica
    'ContractFolderStatus.ProcurementProject.BudgetAmount.EstimatedOverallContractAmount', # Presupuesto
    'ContractFolderStatus.ContractFolderID', #Expediente
    'title', # PLACE title (objective)
    'ContractFolderStatus.ValidNoticeInfo.AdditionalPublicationStatus.AdditionalPublicationDocumentReference.IssueDate', # Date
    'origin',
    'place_id',
    'link'
]


In [6]:
df_PLACE_out = df_PLACE_out[columns_keep_out]
df_PLACE_ins = df_PLACE_ins[columns_keep_ins_min]
df_PLACE_min = df_PLACE_min[columns_keep_ins_min]

In [7]:
df_PLACE_all = pd.concat([df_PLACE_min, df_PLACE_ins, df_PLACE_out], ignore_index=True)

In [8]:
df_PLACE_all['date'] = df_PLACE_all['ContractFolderStatus.ValidNoticeInfo.AdditionalPublicationStatus.AdditionalPublicationDocumentReference.IssueDate'].astype(str)
df_PLACE_all['date'] = df_PLACE_all['date'].str.strip('[]').str.strip('"[').str.split(' ')
df_PLACE_all['date'] = df_PLACE_all['date'].apply(lambda x: str(x[0].strip().strip("'").strip("',").strip('"]') if isinstance(x, list) else x.strip().strip("',").strip('"]')))

In [9]:
df_PLACE_all = df_PLACE_all.rename(columns={
    'ContractFolderStatus.ProcurementProject.RequiredCommodityClassification.ItemClassificationCode':'cpv',
    'ContractFolderStatus.ProcurementProject.RealizedLocation.CountrySubentityCode':'nuts',
    'ContractFolderStatus.LocatedContractingParty.ParentLocatedParty.PartyName.Name':'org_loc',
    'ContractFolderStatus.ProcurementProject.BudgetAmount.EstimatedOverallContractAmount':'budget',
    'ContractFolderStatus.TenderResult.SMEAwardedIndicator':'is_pyme',
    'ContractFolderStatus.ContractFolderID':'expedient',
    'title':'objective',
    'ContractFolderStatus.ValidNoticeInfo.AdditionalPublicationStatus.AdditionalPublicationDocumentReference.IssueDate':'place_date',
    'origin':'origin',
    'place_id':'place_id',
    'link':'link'
})

df_PLACE_all.drop(columns=["is_pyme"], inplace=True)

In [10]:
import numpy as np
import ast

def extract_numbers(x):
    try:
        
        # If x is a string, try to parse it as a list
        if isinstance(x, np.ndarray):
            parsed = ast.literal_eval(x[0])  # Convert string to Python list
            
            # Ensure parsed value is a list
            if isinstance(parsed, list):
                cleaned_list = [float(i) for i in parsed if i != 'nan']  # Convert numbers, filter 'nan'
                return cleaned_list if cleaned_list else np.nan  # Return NaN if empty
            elif isinstance(parsed, float):
                return [parsed]
        
    except (ValueError, SyntaxError, TypeError):
        return np.nan  # Handle invalid cases by returning NaN

In [11]:
# Apply the function to the "cpv" column
df_PLACE_all["cpv"] = df_PLACE_all["cpv"].apply(extract_numbers)

In [12]:
# sort by date
df_PLACE_all = df_PLACE_all.sort_values(by="date", ascending=False)

# add unique id
df_PLACE_all["id"] = range(len(df_PLACE_all))
# sort the columns so the order is "id", "place_id", "origin", "expedient", "objective", "cpv", "nuts", "org_loc", "budget", "date", "link"
df_PLACE_all = df_PLACE_all[["id", "place_id", "origin", "expedient", "objective", "cpv", "nuts", "org_loc", "budget", "date", "place_date", "link"]]


#df_PLACE_all.to_parquet('/export/usuarios_ml4ds/lbartolome/NextProcurement/sproc/place_feb_21/insiders_outsiders_minors_26feb.parquet')

In [13]:
df_PLACE_all.reset_index(drop=True)

,id,place_id,origin,expedient,objective,cpv,nuts,org_loc,budget,date,place_date,link
0,0,https://contrataciondelestado.es/sindicacion/P...,out,2019_213,L'objecte d'aquest encàrrec per part del CONSO...,NaN,ES512,Entitats de l'administració local,NaN,nan,[nan],https://contractaciopublica.cat/ca/detall-publ...
1,1,https://contrataciondelestado.es/sindicacion/l...,ins,DCG-307/2019,Proyecto De Adecuación De Oficinas De Los Serv...,[45000000.0],ES300,Aena. Dirección de Contratación,NaN,nan,[nan],https://contrataciondelestado.es/wps/poc?uri=d...
2,2,https://contrataciondelestado.es/sindicacion/l...,ins,MAD-136/2019,Plan De Obsolescencia Sai´S T123 Fase i,[31000000.0],ES300,Aena. Dirección del Aeropuerto de Madrid-Baraj...,NaN,nan,[nan],https://contrataciondelestado.es/wps/poc?uri=d...
3,3,https://contrataciondelestado.es/sindicacion/l...,ins,20194089-V,Contrato de suministro para la adquisición del...,[18100000.0],ES3,<NA>,NaN,nan,[nan],https://contrataciondelestado.es/wps/poc?uri=d...
4,4,https://contrataciondelestado.es/sindicacion/l...,ins,07/19,Mediación de Riesgos y Seguros.,[66000000.0],ES620,<NA>,NaN,nan,[nan],https://contrataciondelestado.es/wps/poc?uri=d...
...,...,...,...,...,...,...,...,...,...,...,...,...
3827743,3827743,https://contrataciondelestado.es/sindicacion/l...,ins,01/18,Suministro de material de oficina básico con d...,[30197000.0],ES30,<NA>,53146.0,0018-03-02,"[['0018-03-02', '0018-03-02', '2018-03-20'], [...",https://contrataciondelestado.es/wps/poc?uri=d...
3827744,3827744,https://contrataciondelestado.es/sindicacion/P...,out,A2018/000249,Explotación de la cafetería-restaurante y serv...,[55300000.0],ES413,<NA>,54964.8,0002-12-02,"[['0002-12-02', '2017-12-29', '2017-12-29']]",https://contratacion.jcyl.es/web/jcyl/Contrata...
3827745,3827745,https://contrataciondelestado.es/sindicacion/P...,out,A2018/000262,Explotación de la cafetería-restaurante y serv...,[55300000.0],ES413,<NA>,54964.8,0002-12-02,"[['0002-12-02', '2017-12-27', '2017-12-27']]",https://contratacion.jcyl.es/web/jcyl/Contrata...
3827746,3827746,https://contrataciondelestado.es/sindicacion/P...,out,D2018/001233,Suministro de gas natural canalizado en divers...,[9123000.0],ES41,<NA>,255500.0,0002-12-02,"[['0002-12-02', '2018-03-20', '2018-03-21']]",https://contratacion.jcyl.es/web/jcyl/Contrata...


In [14]:
df_PLACE_all[df_PLACE_all["cpv"].isna()]

,id,place_id,origin,expedient,objective,cpv,nuts,org_loc,budget,date,place_date,link
3705619,0,https://contrataciondelestado.es/sindicacion/P...,out,2019_213,L'objecte d'aquest encàrrec per part del CONSO...,NaN,ES512,Entitats de l'administració local,NaN,nan,[nan],https://contractaciopublica.cat/ca/detall-publ...
2753653,28,https://contrataciondelestado.es/sindicacion/l...,ins,251/2018,Enajenación vivienda municipal Plaza Constituc...,NaN,ES418,<NA>,10959.89,nan,[nan],https://contrataciondelestado.es/wps/poc?uri=d...
2754852,47,https://contrataciondelestado.es/sindicacion/l...,ins,2347/2019,Autorización de instalación de postos de venda...,NaN,ES111,<NA>,NaN,nan,[nan],https://contrataciondelestado.es/wps/poc?uri=d...
2767517,168,https://contrataciondelestado.es/sindicacion/l...,ins,838/2019,Iluminación extraordinaria con motivo de event...,NaN,ES611,<NA>,90909.08,nan,[nan],https://contrataciondelestado.es/wps/poc?uri=d...
2733328,234,https://contrataciondelestado.es/sindicacion/l...,ins,374/2019,Enajenación 6 casas de maestros en Calamocha,NaN,ES242,<NA>,202053.50,nan,[nan],https://contrataciondelestado.es/wps/poc?uri=d...
...,...,...,...,...,...,...,...,...,...,...,...,...
2706412,3826857,https://contrataciondelestado.es/sindicacion/l...,ins,124-3299/2014,Autorización uso de dominio público para la co...,NaN,ES,<NA>,NaN,2015-08-27,"[['2015-08-27', '2015-08-27', '2018-11-15']]",https://contrataciondelestado.es/wps/poc?uri=d...
3731189,3827167,https://contrataciondelestado.es/sindicacion/P...,out,AG35/G736/2015/03,Aprofitaments de temporada (GUINGUETES) a les ...,NaN,ES514,Entitats de l'administració local,0.00,2015-06-04,[2015-06-04],https://contractaciopublica.cat/ca/detall-publ...
2922462,3827249,https://contrataciondelestado.es/sindicacion/l...,ins,2/2015,"Concesión para la conservación, reposición, ma...",NaN,ES,<NA>,NaN,2015-04-10',"[['2015-04-10'], ['2015-01-19'], ['2015-01-19'...",https://contrataciondelestado.es/wps/poc?uri=d...
3705700,3827551,https://contrataciondelestado.es/sindicacion/P...,out,VA/2012046,Prestacio del servei de posada en funcionament...,NaN,ES511,Entitats de l'administració local,0.00,2013-06-21,"[['2013-06-21', '2013-06-25'], ['2023-02-27']]",https://contractaciopublica.cat/ca/detall-publ...


In [15]:
#df_PLACE_all.reset_index(drop=True).to_parquet("/export/usuarios_ml4ds/lbartolome/NextProcurement/sproc/place_feb_21/place_all.parquet")
df_PLACE_all_processed = pd.read_parquet("/export/usuarios_ml4ds/lbartolome/NextProcurement/sproc/place_feb_21/preprocessed/place_all_embeddings.parquet")

In [16]:
df_PLACE_all_processed_augmented = pd.merge(df_PLACE_all, df_PLACE_all_processed, on="id", how="left")

In [17]:
# save file with all metadata as well
df_PLACE_all_processed_augmented.to_parquet("/export/usuarios_ml4ds/lbartolome/NextProcurement/sproc/place_feb_21/place_all_embeddings_metadata.parquet")

In [18]:
df_PLACE_all_processed_augmented

,id,place_id,origin,expedient,objective,cpv,nuts,org_loc,budget,date,place_date,link,raw_text,lemmas,embeddings
0,0,https://contrataciondelestado.es/sindicacion/P...,out,2019_213,L'objecte d'aquest encàrrec per part del CONSO...,NaN,ES512,Entitats de l'administració local,NaN,nan,[nan],https://contractaciopublica.cat/ca/detall-publ...,L'objecte d'aquest encàrrec per part del CONSO...,consorci_vies verdes_girona tragsa conservació...,-0.23268852 -0.2679665 -0.072283626 -0.1170956...
1,1,https://contrataciondelestado.es/sindicacion/l...,ins,DCG-307/2019,Proyecto De Adecuación De Oficinas De Los Serv...,[45000000.0],ES300,Aena. Dirección de Contratación,NaN,nan,[nan],https://contrataciondelestado.es/wps/poc?uri=d...,Proyecto De Adecuación De Oficinas De Los Serv...,proyecto adecuación oficinas servicios_central...,-0.02814787 -0.13409117 -0.06640905 -0.0788602...
2,2,https://contrataciondelestado.es/sindicacion/l...,ins,MAD-136/2019,Plan De Obsolescencia Sai´S T123 Fase i,[31000000.0],ES300,Aena. Dirección del Aeropuerto de Madrid-Baraj...,NaN,nan,[nan],https://contrataciondelestado.es/wps/poc?uri=d...,Plan De Obsolescencia Sai´S T123 Fase i,plan obsolescencia fase,-0.23452374 0.23028997 0.18730758 -0.19488867 ...
3,3,https://contrataciondelestado.es/sindicacion/l...,ins,20194089-V,Contrato de suministro para la adquisición del...,[18100000.0],ES3,<NA>,NaN,nan,[nan],https://contrataciondelestado.es/wps/poc?uri=d...,Contrato de suministro para la adquisición del...,suministro adquisición vestuario equipo protec...,-0.12146915 0.13177831 -0.066414416 0.2246932 ...
4,4,https://contrataciondelestado.es/sindicacion/l...,ins,07/19,Mediación de Riesgos y Seguros.,[66000000.0],ES620,<NA>,NaN,nan,[nan],https://contrataciondelestado.es/wps/poc?uri=d...,Mediación de Riesgos y Seguros.,mediación_riesgos seguros,-0.020959144 0.1747884 -0.2840229 0.052032873 ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3827743,3827743,https://contrataciondelestado.es/sindicacion/l...,ins,01/18,Suministro de material de oficina básico con d...,[30197000.0],ES30,<NA>,53146.0,0018-03-02,"[['0018-03-02', '0018-03-02', '2018-03-20'], [...",https://contrataciondelestado.es/wps/poc?uri=d...,Suministro de material de oficina básico con d...,suministro material oficina básico destino alm...,0.053728275 0.033718243 0.010642192 0.06366115...
3827744,3827744,https://contrataciondelestado.es/sindicacion/P...,out,A2018/000249,Explotación de la cafetería-restaurante y serv...,[55300000.0],ES413,<NA>,54964.8,0002-12-02,"[['0002-12-02', '2017-12-29', '2017-12-29']]",https://contratacion.jcyl.es/web/jcyl/Contrata...,Explotación de la cafetería-restaurante y serv...,explotación servicio comedor usuario servicio ...,0.19895372 -0.22709867 -0.14535315 0.14950758 ...
3827745,3827745,https://contrataciondelestado.es/sindicacion/P...,out,A2018/000262,Explotación de la cafetería-restaurante y serv...,[55300000.0],ES413,<NA>,54964.8,0002-12-02,"[['0002-12-02', '2017-12-27', '2017-12-27']]",https://contratacion.jcyl.es/web/jcyl/Contrata...,Explotación de la cafetería-restaurante y serv...,explotación servicio comedor usuario servicio ...,0.18068197 -0.22375913 -0.15644608 0.17646262 ...
3827746,3827746,https://contrataciondelestado.es/sindicacion/P...,out,D2018/001233,Suministro de gas natural canalizado en divers...,[9123000.0],ES41,<NA>,255500.0,0002-12-02,"[['0002-12-02', '2018-03-20', '2018-03-21']]",https://contratacion.jcyl.es/web/jcyl/Contrata...,Suministro de gas natural canalizado en divers...,suministro natural_canalizado centros consejer...,0.051175013 -0.06626684 -0.016259827 0.0174854...


In [19]:
columns_keep = ['id', 'place_id', 'origin', 'objective', 'cpv', 'date', 'link', 'raw_text', 'lemmas', 'embeddings']
df_PLACE_all_processed_augmented_for_np_backend = df_PLACE_all_processed_augmented.copy()[columns_keep]

# rename 'objective' to 'title'
df_PLACE_all_processed_augmented_for_np_backend = df_PLACE_all_processed_augmented_for_np_backend.rename(columns={"objective":"title"})
#df_PLACE_all_processed_augmented_for_np_backend.to_parquet("/export/usuarios_ml4ds/lbartolome/NextProcurement/sproc/place_feb_21/place_all_embeddings_for_np_backend.parquet")

In [20]:
df_PLACE_all_processed_augmented["embeddings"]

0          -0.23268852 -0.2679665 -0.072283626 -0.1170956...
1          -0.02814787 -0.13409117 -0.06640905 -0.0788602...
2          -0.23452374 0.23028997 0.18730758 -0.19488867 ...
3          -0.12146915 0.13177831 -0.066414416 0.2246932 ...
4          -0.020959144 0.1747884 -0.2840229 0.052032873 ...
                                 ...                        
3827743    0.053728275 0.033718243 0.010642192 0.06366115...
3827744    0.19895372 -0.22709867 -0.14535315 0.14950758 ...
3827745    0.18068197 -0.22375913 -0.15644608 0.17646262 ...
3827746    0.051175013 -0.06626684 -0.016259827 0.0174854...
3827747    -0.31032175 0.08248795 0.19682516 -0.16974026 ...
Name: embeddings, Length: 3827748, dtype: object

In [21]:
import pandas as pd
df_PLACE_all_processed_augmented = pd.read_parquet("/export/usuarios_ml4ds/lbartolome/NextProcurement/sproc/place_feb_21/place_all_embeddings_metadata.parquet").sample(n=10)

In [31]:
df_PLACE_all_processed_augmented_for_np_backend[df_PLACE_all_processed_augmented_for_np_backend.place_id == "https://contrataciondelestado.es/sindicacion/licitacionesPerfilContratante/10968096"]

,id,place_id,origin,title,cpv,date,link,raw_text,lemmas,embeddings
1752955,1752955,https://contrataciondelestado.es/sindicacion/l...,ins,El objeto de la presente contratación es el su...,[9310000.0],2022-08-10,https://contrataciondelestado.es/wps/poc?uri=d...,El objeto de la presente contratación es el su...,presente suministro energía_eléctrico punto su...,0.014003306 0.10802064 0.09031655 0.029915154 ...


In [22]:
partition = df_PLACE_all_processed_augmented[["id","embeddings"]].copy()
partition

,id,embeddings
3092137,3092137,-0.06400152 0.20049936 0.06676818 -0.07143328 ...
1111633,1111633,0.21896419 -0.005621305 -0.08529077 0.04404995...
1924974,1924974,-0.038822655 0.14823373 0.21785323 0.015813166...
3485437,3485437,-0.16780095 -0.16992909 0.070153505 -0.1584008...
1691463,1691463,-0.12178727 0.4067 -0.20540686 -0.010200002 0....
1632036,1632036,-0.12017949 0.2928279 -0.08681707 0.023188967 ...
2253249,2253249,-0.105017856 -0.01121879 0.08896475 0.1169358 ...
2666714,2666714,-0.06128742 0.11697567 -0.24811102 -0.20948139...
2343679,2343679,0.03280621 0.038329627 0.1159968 -0.023392621 ...
2351793,2351793,0.5173332 -0.00043927642 0.112181924 -0.082452...


In [23]:
x = partition.iloc[0]
print(x)

id                                                      3092137
embeddings    -0.06400152 0.20049936 0.06676818 -0.07143328 ...
Name: 3092137, dtype: object


In [24]:
x_parse = [float(val) for val in x.split()] if isinstance(x, str) else  (x if isinstance(x, list) else [])
x_parse[:1000]

[]

In [25]:
import numpy as np
partition["embeddings"] =  partition["embeddings"].apply(lambda x: [float(val) for _, val in enumerate(x.split())])

In [26]:
records =  partition.to_dict(orient="records")
print(records[0])

{'id': 3092137, 'embeddings': [-0.06400152, 0.20049936, 0.06676818, -0.07143328, -0.08047458, 0.002053371, 0.07716137, 0.22020456, -0.28247023, -0.017101917, 0.010771148, 0.047084928, 0.16610555, -0.04576753, -0.2915086, 0.05218185, 0.23766086, -0.0155741265, 0.17861196, -0.06815017, 0.19968055, -0.043139342, -0.09216876, -0.262776, -0.031837817, 0.043755025, -0.2013371, -0.12149948, 0.05266814, -0.13283099, -0.23269874, 0.1094389, 0.121753216, -0.14007698, 0.27479187, -0.104867786, 0.026929157, -0.15017192, 0.102635965, 0.01929169, -0.24135247, -0.4131108, -0.2509707, 0.13358371, 0.0036846327, -0.028042855, 0.06293427, -0.062706776, -0.42669758, -0.1578978, 0.37231034, -0.13789706, 0.12582086, -0.033395007, -0.119123265, -0.18004313, 0.36157325, -0.15269396, -0.42619303, -0.035767388, 0.07963405, -0.06864183, -0.29580468, 0.048076555, -0.030662479, 0.17614579, 0.059282355, -0.1377151, -0.14743508, -0.0057752826, -0.1738875, -0.22019802, -0.16094348, -0.09451888, -0.07674188, 0.1451194